#### Generate files

Generate level, group, ref_team, ref_conflict, ref_available, team_available.

In [2]:
import pandas as pd
import random
import os
import math

# Create 'cases' folder if it doesn't exist
if not os.path.exists("cases"):
    os.makedirs("cases")

# List of NTU departments in Mandarin
departments = [
    "中國文學系", "外國語文學系", "歷史學系", "哲學系", "人類學系", "圖書資訊學系", "日本語文學系", "戲劇學系",
    "數學系", "物理學系", "化學系", "地質科學系", "大氣科學系", "地理環境資源學系", "心理學系",
    "經濟學系", "社會學系", "政治學系", "社會工作學系",
    "醫學系", "牙醫學系", "藥學系", "醫學檢驗暨生物技術學系", "護理學系", "物理治療學系", "職能治療學系",
    "土木工程學系", "機械工程學系", "化學工程學系", "工程科學及海洋工程學系", "材料科學與工程學系",
    "電機工程學系", "資訊工程學系",
    "農藝學系", "生物環境系統工程學系", "農業化學系", "森林環境暨資源學系", "動物科學技術學系",
    "農業經濟學系", "園藝學系", "獸醫學系", "生物產業傳播暨發展學系", "昆蟲學系",
    "工商管理學系", "會計學系", "財務金融學系", "國際企業學系", "資訊管理學系",
    "公共衛生學系",
    "法律學系",
    "生命科學系", "生化科技學系",
    "應用數學科學研究所", "統計與數據科學研究所", "海洋研究所", "天文物理研究所",
    "生物化學及分子生物學系", "微生物學系", "藥理學系", "植物科學研究所", "建築與城鄉研究所"
]

# List of referee names
referees = [
    "小馬", "恩臨", "羿君", "茵茵", "芳芳", "阿冠", "小魚", "yoyo", "家葳", "絲瓜",
    "大餅", "手槍", "阿程", "阿侑", "阿宛"
]

# Function to generate data for the 'level' sheet (for files 1-4)
def generate_level_data(num_rows, level_distribution=None):
    # If no specific level distribution, spread evenly across levels 1-4
    if level_distribution is None:
        base_count = num_rows // 4
        remainder = num_rows % 4
        levels = ([1] * base_count + [2] * base_count + [3] * base_count + [4] * base_count +
                  [i + 1 for i in range(remainder)])
    else:
        levels = []
        for level, count in enumerate(level_distribution, 1):
            levels.extend([level] * count)
    
    # Shuffle departments to avoid repetition patterns
    available_depts = random.sample(departments * (num_rows // len(departments) + 1), num_rows)
    
    # Create data
    data = {
        "team_name": available_depts[:num_rows],
        "level": levels
    }
    
    # Shuffle to randomize order
    df = pd.DataFrame(data)
    df = df.sample(frac=1).reset_index(drop=True)
    return df

# Function to generate data for the 'level' sheet for case_5 (40 teams, 3 departments with 2 teams)
def generate_level_data_case5():
    # Select 3 departments for double teams, including 土木工程學系
    double_depts = random.sample([d for d in departments if d != "土木工程學系"], 2) + ["土木工程學系"]
    
    # Create team names for double departments (A and B teams)
    team_names = []
    levels = []
    for dept in double_depts:
        team_names.append(f"{dept}A")
        team_names.append(f"{dept}B")
        # Assign A team a better level (1 or 2) and B team a worse level (3 or 4)
        a_level = random.choice([1, 2])
        b_level = random.choice([3, 4])
        levels.extend([a_level, b_level])
    
    # Select 34 unique departments for the remaining teams
    remaining_depts = random.sample([d for d in departments if d not in double_depts], 34)
    team_names.extend(remaining_depts)
    
    # Assign remaining levels to achieve even distribution (10 per level for 40 teams)
    remaining_levels = []
    target_per_level = 10  # 40 teams / 4 levels
    current_counts = {1: levels.count(1), 2: levels.count(2), 3: levels.count(3), 4: levels.count(4)}
    for level in range(1, 5):
        needed = target_per_level - current_counts.get(level, 0)
        remaining_levels.extend([level] * needed)
    
    # Combine and shuffle
    levels.extend(remaining_levels)
    data = {
        "team_name": team_names,
        "level": levels
    }
    df = pd.DataFrame(data)
    df = df.sample(frac=1).reset_index(drop=True)
    return df

# Function to generate data for the 'ref_team' sheet
def generate_ref_team_data():
    # Randomly assign departments to referees
    assigned_depts = random.sample(departments * (len(referees) // len(departments) + 1), len(referees))
    data = {
        "ref_name": referees,
        "team_name": assigned_depts
    }
    df = pd.DataFrame(data)
    return df

# Function to generate data for the 'group' sheet
def generate_group_data(level_df, case_number=1):
    num_teams = len(level_df)
    n = num_teams // 4
    remainder = num_teams % 4
    
    # Determine number of four-team and three-team groups
    if remainder == 0:  # 4n teams
        num_four_team_groups = n
        num_three_team_groups = 0
    elif remainder == 1:  # 4n+1 teams
        num_four_team_groups = n - 2
        num_three_team_groups = 3
    elif remainder == 2:  # 4n+2 teams
        num_four_team_groups = n - 1
        num_three_team_groups = 2
    else:  # 4n+3 teams
        num_four_team_groups = n
        num_three_team_groups = 1
    
    # Initialize groups
    groups = []
    group_labels = [chr(65 + i) for i in range(num_four_team_groups + num_three_team_groups)]  # A, B, C, ...
    
    # Create a team-to-level mapping for the level column
    team_to_level = dict(zip(level_df['team_name'], level_df['level']))
    
    # Group teams by level
    teams_by_level = {i: level_df[level_df['level'] == i]['team_name'].tolist() for i in range(1, 5)}
    
    # For case_5, identify A/B team pairs to avoid placing them in the same group
    ab_pairs = {}
    if case_number == 5:
        for team in level_df['team_name']:
            if team.endswith('A') or team.endswith('B'):
                base_dept = team[:-1]
                ab_pairs.setdefault(base_dept, []).append(team)
    
    # Create four-team groups (one team per level)
    for i in range(num_four_team_groups):
        group = []
        used_depts = set()
        for level in range(1, 5):
            if teams_by_level[level]:
                # For case_5, ensure no A/B pairs in the same group
                if case_number == 5:
                    available_teams = [
                        t for t in teams_by_level[level]
                        if not any(
                            base_dept in ab_pairs and
                            t != ab_pairs[base_dept][0] and
                            ab_pairs[base_dept][0] in group
                            for base_dept in ab_pairs
                        )
                    ]
                    if not available_teams:
                        available_teams = teams_by_level[level]  # Fallback if no valid teams
                else:
                    available_teams = teams_by_level[level]
                
                if available_teams:
                    team = random.choice(available_teams)
                    teams_by_level[level].remove(team)
                    group.append(team)
                    # Track base department for case_5
                    if case_number == 5:
                        base_dept = team[:-1] if team.endswith(('A', 'B')) else team
                        used_depts.add(base_dept)
        groups.append((group_labels[i], group))
    
    # Create three-team groups (maximize level diversity)
    remaining_teams = []
    for level in teams_by_level:
        remaining_teams.extend([(team, level) for team in teams_by_level[level]])
    
    for i in range(num_three_team_groups):
        group = []
        used_depts = set()
        # Try to pick 3 teams with different levels
        level_counts = {i: len(teams_by_level[i]) for i in range(1, 5)}
        available_levels = sorted([l for l in level_counts if level_counts[l] > 0], key=lambda x: level_counts[x], reverse=True)
        
        # Select up to 3 different levels
        selected_teams = []
        for level in available_levels:
            if len(selected_teams) < 3 and teams_by_level[level]:
                # For case_5, ensure no A/B pairs in the same group
                if case_number == 5:
                    available_teams = [
                        t for t in teams_by_level[level]
                        if not any(
                            base_dept in ab_pairs and
                            t != ab_pairs[base_dept][0] and
                            ab_pairs[base_dept][0] in group
                            for base_dept in ab_pairs
                        )
                    ]
                else:
                    available_teams = teams_by_level[level]
                
                if available_teams:
                    team = random.choice(available_teams)
                    teams_by_level[level].remove(team)
                    selected_teams.append(team)
                    if case_number == 5:
                        base_dept = team[:-1] if team.endswith(('A', 'B')) else team
                        used_depts.add(base_dept)
        
        # If fewer than 3 teams, fill with remaining teams
        while len(selected_teams) < 3 and remaining_teams:
            available_teams = [
                (t, l) for t, l in remaining_teams
                if case_number != 5 or not any(
                    base_dept in ab_pairs and
                    t != ab_pairs[base_dept][0] and
                    ab_pairs[base_dept][0] in selected_teams
                    for base_dept in ab_pairs
                )
            ]
            if available_teams:
                team, level = random.choice(available_teams)
                selected_teams.append(team)
                teams_by_level[level].remove(team)
                remaining_teams = [(t, l) for t, l in remaining_teams if t != team]
        
        groups.append((group_labels[num_four_team_groups + i], selected_teams))
    
    # Create DataFrame for group sheet with level column
    group_data = []
    for group_label, group_teams in groups:
        for team in group_teams:
            group_data.append({
                "group": group_label,
                "team_name": team,
                "level": team_to_level[team]
            })
    
    df = pd.DataFrame(group_data)
    return df, group_labels

# Function to generate data for the 'ref_conflicts' sheet
def generate_ref_conflicts_data(ref_team_df, group_df, group_labels, case_number=1):
    # Create a mapping of team names to groups
    group_to_teams = group_df.groupby('group')['team_name'].apply(list).to_dict()
    
    # For case_5, identify A/B team pairs
    ab_pairs = {}
    if case_number == 5:
        for team in group_df['team_name']:
            if team.endswith('A') or team.endswith('B'):
                base_dept = team[:-1]
                ab_pairs.setdefault(base_dept, []).append(team)
    
    # Initialize conflicts matrix
    conflicts_data = []
    for ref_name in referees:
        ref_dept = ref_team_df[ref_team_df['ref_name'] == ref_name]['team_name'].iloc[0]
        row = {"ref_name": ref_name}
        for group_label in group_labels:
            group_teams = group_to_teams.get(group_label, [])
            # Check for conflict
            conflict = False
            # Standard conflict: referee's department is in the group
            if ref_dept in group_teams:
                conflict = True
            # For case_5: if referee's department has A/B teams, check both
            if case_number == 5 and ref_dept in ab_pairs:
                if any(team in group_teams for team in ab_pairs[ref_dept]):
                    conflict = True
            row[group_label] = 0 if conflict else 1
        conflicts_data.append(row)
    
    df = pd.DataFrame(conflicts_data)
    return df

# Function to generate data for the 'ref_available' sheet
def generate_ref_available_data():
    days = 70
    pattern_length = 5
    midterm_start = 31  # Days 31-35 are midterm week
    midterm_end = 35
    availability_data = []
    
    for ref_name in referees:
        # Generate a 5-day pattern with weighted random choices
        pattern = [random.choices([1, 0.5, 0], weights=[0.3, 0.5, 0.2], k=1)[0] for _ in range(pattern_length)]
        # Extend pattern to 70 days (14 cycles)
        availability = pattern * (days // pattern_length)
        # Set zeros for days 31-35 (midterm week)
        for i in range(midterm_start - 1, midterm_end):
            availability[i] = 0
        
        row = {"ref_name": ref_name}
        for day in range(1, days + 1):
            row[str(day)] = availability[day - 1]  # Use numerical string for column name
        availability_data.append(row)
    
    df = pd.DataFrame(availability_data)
    return df

# Function to generate data for the 'team_available' sheet
def generate_team_available_data(level_df):
    days = ["mon", "tue", "wed", "thur", "fri"]
    availability_data = []
    
    for team_name in level_df['team_name']:
        # Randomly select 0 to 2 days to be unavailable (0)
        num_unavailable = random.randint(0, 2)
        unavailable_days = random.sample(days, num_unavailable)
        # Create availability row
        row = {"team_name": team_name}
        for day in days:
            row[day] = 0 if day in unavailable_days else 1
        availability_data.append(row)
    
    df = pd.DataFrame(availability_data)
    return df

# Generate and save the five files
# File 1: 16 rows, 4 per level, with all sheets
data1_level = generate_level_data(16, [4, 4, 4, 4])
data1_ref = generate_ref_team_data()
data1_group, group_labels1 = generate_group_data(data1_level, case_number=1)
data1_conflicts = generate_ref_conflicts_data(data1_ref, data1_group, group_labels1, case_number=1)
data1_available = generate_ref_available_data()
data1_team_available = generate_team_available_data(data1_level)
with pd.ExcelWriter("cases/case_1.xlsx", engine="openpyxl") as writer:
    data1_level.to_excel(writer, sheet_name="level", index=False)
    data1_ref.to_excel(writer, sheet_name="ref_team", index=False)
    data1_group.to_excel(writer, sheet_name="group", index=False)
    data1_conflicts.to_excel(writer, sheet_name="ref_conflicts", index=False)
    data1_available.to_excel(writer, sheet_name="ref_available", index=False)
    data1_team_available.to_excel(writer, sheet_name="team_available", index=False)

# File 2: 38 rows, even distribution, with all sheets
data2_level = generate_level_data(38)
data2_ref = generate_ref_team_data()
data2_group, group_labels2 = generate_group_data(data2_level, case_number=2)
data2_conflicts = generate_ref_conflicts_data(data2_ref, data2_group, group_labels2, case_number=2)
data2_available = generate_ref_available_data()
data2_team_available = generate_team_available_data(data2_level)
with pd.ExcelWriter("cases/case_2.xlsx", engine="openpyxl") as writer:
    data2_level.to_excel(writer, sheet_name="level", index=False)
    data2_ref.to_excel(writer, sheet_name="ref_team", index=False)
    data2_group.to_excel(writer, sheet_name="group", index=False)
    data2_conflicts.to_excel(writer, sheet_name="ref_conflicts", index=False)
    data2_available.to_excel(writer, sheet_name="ref_available", index=False)
    data2_team_available.to_excel(writer, sheet_name="team_available", index=False)

# File 3: 25 rows, even distribution, with all sheets
data3_level = generate_level_data(25)
data3_ref = generate_ref_team_data()
data3_group, group_labels3 = generate_group_data(data3_level, case_number=3)
data3_conflicts = generate_ref_conflicts_data(data3_ref, data3_group, group_labels3, case_number=3)
data3_available = generate_ref_available_data()
data3_team_available = generate_team_available_data(data3_level)
with pd.ExcelWriter("cases/case_3.xlsx", engine="openpyxl") as writer:
    data3_level.to_excel(writer, sheet_name="level", index=False)
    data3_ref.to_excel(writer, sheet_name="ref_team", index=False)
    data3_group.to_excel(writer, sheet_name="group", index=False)
    data3_conflicts.to_excel(writer, sheet_name="ref_conflicts", index=False)
    data3_available.to_excel(writer, sheet_name="ref_available", index=False)
    data3_team_available.to_excel(writer, sheet_name="team_available", index=False)

# File 4: 31 rows, even distribution, with all sheets
data4_level = generate_level_data(31)
data4_ref = generate_ref_team_data()
data4_group, group_labels4 = generate_group_data(data4_level, case_number=4)
data4_conflicts = generate_ref_conflicts_data(data4_ref, data4_group, group_labels4, case_number=4)
data4_available = generate_ref_available_data()
data4_team_available = generate_team_available_data(data4_level)
with pd.ExcelWriter("cases/case_4.xlsx", engine="openpyxl") as writer:
    data4_level.to_excel(writer, sheet_name="level", index=False)
    data4_ref.to_excel(writer, sheet_name="ref_team", index=False)
    data4_group.to_excel(writer, sheet_name="group", index=False)
    data4_conflicts.to_excel(writer, sheet_name="ref_conflicts", index=False)
    data4_available.to_excel(writer, sheet_name="ref_available", index=False)
    data4_team_available.to_excel(writer, sheet_name="team_available", index=False)

# File 5: 40 rows, 3 departments with 2 teams, with all sheets
data5_level = generate_level_data_case5()
data5_ref = generate_ref_team_data()
data5_group, group_labels5 = generate_group_data(data5_level, case_number=5)
data5_conflicts = generate_ref_conflicts_data(data5_ref, data5_group, group_labels5, case_number=5)
data5_available = generate_ref_available_data()
data5_team_available = generate_team_available_data(data5_level)
with pd.ExcelWriter("cases/case_5.xlsx", engine="openpyxl") as writer:
    data5_level.to_excel(writer, sheet_name="level", index=False)
    data5_ref.to_excel(writer, sheet_name="ref_team", index=False)
    data5_group.to_excel(writer, sheet_name="group", index=False)
    data5_conflicts.to_excel(writer, sheet_name="ref_conflicts", index=False)
    data5_available.to_excel(writer, sheet_name="ref_available", index=False)
    data5_team_available.to_excel(writer, sheet_name="team_available", index=False)

print("Generated 5 XLSX files in the 'cases' folder:")
print("- case_1.xlsx to case_5.xlsx: 'level', 'ref_team', 'group', 'ref_conflicts', 'ref_available', and 'team_available' sheets")
print("- case_5.xlsx: 'level' sheet with 40 teams (3 departments with A/B teams, A/B not in same group)")
print("- 'group' sheet: includes 'group', 'team_name', and 'level' columns")
print("- 'ref_conflicts' sheet: 1 (can ref) or 0 (cannot ref) for each referee and group")
print("- 'ref_available' sheet: 1 (available), 0.5 (uncertain), or 0 (unavailable) for each referee and days 1-70, with 5-day pattern, zeros for days 31-35")
print("- 'team_available' sheet: 1 (available) or 0 (unavailable) for each team and weekday, with at most 2 unavailable days per team")

Generated 5 XLSX files in the 'cases' folder:
- case_1.xlsx to case_5.xlsx: 'level', 'ref_team', 'group', 'ref_conflicts', 'ref_available', and 'team_available' sheets
- case_5.xlsx: 'level' sheet with 40 teams (3 departments with A/B teams, A/B not in same group)
- 'group' sheet: includes 'group', 'team_name', and 'level' columns
- 'ref_conflicts' sheet: 1 (can ref) or 0 (cannot ref) for each referee and group
- 'ref_available' sheet: 1 (available), 0.5 (uncertain), or 0 (unavailable) for each referee and days 1-70, with 5-day pattern, zeros for days 31-35
- 'team_available' sheet: 1 (available) or 0 (unavailable) for each team and weekday, with at most 2 unavailable days per team


### Define heuristic solution

A match needs two teams (in the same group), a referee, a court, and a day.  Two teams are available to match up if both teams are available on that day of the week. The referee of the game has to be available on that day, too. The court should be court 0, 1, 2, or 3. We start scheduling matches from Group A, starting on day 1. If a matchup in Group A is available and hasn’t been played yet, pick the matchup and set it at an available court. If there are no available matchups available within Group A, look within Group B. Then we look in Group C, and so on. If all courts are filled or there exists no match that can be scheduled, we move on to the next day. From the referees whose availability is 1 on that day(and doesn’t have a game scheduled on that day), we assign the referee that has the least amount of matches reffed. If no referees have a 1 on their availability, we pick from the referee that have a 0.5 availability and reffed least games. .If a match is made, we record a few things: how many times have both teams been scheduled that week, and how many games have the referee been scheduled. 

In [22]:
import pandas as pd
import os
from itertools import combinations
import random

# Create 'schedules' folder if it doesn't exist
if not os.path.exists("schedules"):
    os.makedirs("schedules")

# Weekday mapping (5-day cycle: Day 1 = Monday, Day 5 = Friday, Day 6 = Monday)
WEEKDAYS = {1: "mon", 2: "tue", 3: "wed", 4: "thur", 0: "fri"}

def get_weekday(day):
    """Map day number to weekday name (5-day cycle)."""
    return WEEKDAYS[day % 5]

def get_week_number(day):
    """Determine the week number (1-based) for a given day (7-day weeks)."""
    return (day - 1) // 7 + 1

def generate_matches(group_df):
    """Generate all possible matches per group."""
    matches = []
    groups = group_df['group'].unique()
    for group in groups:
        teams = group_df[group_df['group'] == group]['team_name'].tolist()
        for team1, team2 in combinations(teams, 2):
            matches.append({
                'group': group,
                'team1': team1,
                'team2': team2,
                'match_str': f"{team1} vs {team2}",
                'scheduled': False
            })
    return matches

def find_available_referee(ref_available, ref_conflicts, group, day, referee_day_assignments, referee_counts):
    """Find an available referee for a match on a given day, checking conflicts."""
    day_str = str(day)
    available_refs = []
    
    # Validate group exists in ref_conflicts
    if group not in ref_conflicts.columns:
        return None
    
    # Try referees with availability 1
    for _, ref in ref_available.iterrows():
        ref_name = ref['ref_name']
        # Check: no conflict (1 in ref_conflicts), available (1), not assigned
        if (ref_conflicts[ref_conflicts['ref_name'] == ref_name][group].iloc[0] == 1 and
            ref[day_str] == 1 and
            ref_name not in referee_day_assignments.get(day, [])):
            available_refs.append((ref_name, referee_counts.get(ref_name, 0)))
    
    # If none, try referees with availability 0.5
    if not available_refs:
        for _, ref in ref_available.iterrows():
            ref_name = ref['ref_name']
            # Check: no conflict (1 in ref_conflicts), available (0.5), not assigned
            if (ref_conflicts[ref_conflicts['ref_name'] == ref_name][group].iloc[0] == 1 and
                ref[day_str] == 0.5 and
                ref_name not in referee_day_assignments.get(day, [])):
                available_refs.append((ref_name, referee_counts.get(ref_name, 0)))
    
    # Select referee with fewest games officiated
    if available_refs:
        available_refs.sort(key=lambda x: x[1])
        return available_refs[0][0]
    return None

def schedule_matches(case_number):
    """Schedule matches for a given case and save to output file."""
    # Load input data
    input_file = f"cases/case_{case_number}.xlsx"
    try:
        level_df = pd.read_excel(input_file, sheet_name="level")
        group_df = pd.read_excel(input_file, sheet_name="group")
        ref_available_df = pd.read_excel(input_file, sheet_name="ref_available")
        team_available_df = pd.read_excel(input_file, sheet_name="team_available")
        ref_conflicts_df = pd.read_excel(input_file, sheet_name="ref_conflicts")
    except Exception as e:
        print(f"Error loading case_{case_number}.xlsx: {e}")
        return -1
    
    # Validate input data
    if ref_available_df.empty or group_df.empty or team_available_df.empty or ref_conflicts_df.empty:
        print(f"Case {case_number}: One or more input DataFrames are empty.")
        return -1
    
    # Generate all possible matches
    matches = generate_matches(group_df)
    groups = sorted(group_df['group'].unique())
    
    # Initialize tracking
    schedule = []
    team_day_assignments = {}  # {day: [teams]}
    team_weekly_counts = {}  # {(week, team): count}
    referee_counts = {}  # {referee: count}
    referee_day_assignments = {}  # {day: [referees]}
    courts = [0, 1, 2, 3]
    last_match_day = 0  # Track the last day a match is scheduled
    half_availability_count = 0  # Track matches with 0.5 availability referees
    
    # Schedule matches
    for day in range(1, 71):
        weekday = get_weekday(day)
        week = get_week_number(day)
        courts_used = []
        matches_scheduled = 0
        
        # Skip days 31-35 (all referees unavailable)
        if 31 <= day <= 35:
            continue
        
        for group in groups:
            if matches_scheduled >= 4:
                break
            
            group_matches = [m for m in matches if m['group'] == group and not m['scheduled']]
            random.shuffle(group_matches)
            
            for match in group_matches:
                if matches_scheduled >= 4:
                    break
                
                team1, team2 = match['team1'], match['team2']
                
                # Check team availability
                try:
                    team1_available = team_available_df[team_available_df['team_name'] == team1][weekday].iloc[0]
                    team2_available = team_available_df[team_available_df['team_name'] == team2][weekday].iloc[0]
                except IndexError:
                    continue
                if team1_available != 1 or team2_available != 1:
                    continue
                
                # Check daily limit
                day_teams = team_day_assignments.get(day, [])
                if team1 in day_teams or team2 in day_teams:
                    continue
                
                # Check weekly limit
                team1_week_count = team_weekly_counts.get((week, team1), 0)
                team2_week_count = team_weekly_counts.get((week, team2), 0)
                if team1_week_count >= 2 or team2_week_count >= 2:
                    continue
                
                # Find referee (checks ref_conflicts)
                referee = find_available_referee(
                    ref_available_df, ref_conflicts_df, group, day,
                    referee_day_assignments, referee_counts
                )
                if not referee:
                    continue
                
                # Assign court
                available_courts = [c for c in courts if c not in courts_used]
                if not available_courts:
                    continue
                court = available_courts[0]
                
                # Check referee availability for penalty
                ref_avail = ref_available_df[ref_available_df['ref_name'] == referee][str(day)].iloc[0]
                if ref_avail == 0.5:
                    half_availability_count += 1
                
                # Schedule match
                schedule.append({
                    'Day': day,
                    'Group': group,
                    'Field': court,
                    'Match': match['match_str'],
                    'Referee': referee
                })
                match['scheduled'] = True
                courts_used.append(court)
                matches_scheduled += 1
                last_match_day = max(last_match_day, day)  # Update last match day
                
                # Update tracking
                team_day_assignments.setdefault(day, []).extend([team1, team2])
                team_weekly_counts[(week, team1)] = team1_week_count + 1
                team_weekly_counts[(week, team2)] = team2_week_count + 1
                referee_counts[referee] = referee_counts.get(referee, 0) + 1
                referee_day_assignments.setdefault(day, []).append(referee)
        
        # Clean up weekly counts
        team_weekly_counts = {k: v for k, v in team_weekly_counts.items() if k[0] == week}
    
    # Check unscheduled matches
    unscheduled = [m['match_str'] for m in matches if not m['scheduled']]
    if unscheduled:
        print(f"Case {case_number}: {len(unscheduled)} matches could not be scheduled: {unscheduled}")
    else:
        print(f"Case {case_number}: All matches scheduled successfully.")
    
    # Print the last match day
    if last_match_day > 0:
        print(f"Case {case_number}: Last match scheduled on day {last_match_day} ({get_weekday(last_match_day)})")
    else:
        print(f"Case {case_number}: No matches were scheduled.")
    
    # Calculate objective function
    makespan = last_match_day
    penalty = 0.1 * half_availability_count
    total_matches = len(matches) - len(unscheduled)
    avg_games = total_matches / len(ref_available_df) if not ref_available_df.empty else 0
    balance_penalty = sum(
        abs(referee_counts.get(r, 0) - avg_games)
        for r in ref_available_df['ref_name']
    ) if not ref_available_df.empty else 0
    objective_value = makespan + penalty + 0.01 * balance_penalty
    print(f"Case {case_number}: Objective value = {objective_value:.4f} "
          f"(makespan={makespan}, penalty={penalty:.4f}, balance_penalty={balance_penalty:.4f})")
    
    # Prepare output
    schedule_df = pd.DataFrame(schedule)
    ref_counts_df = pd.DataFrame({
        'Referee': list(referee_counts.keys()),
        'Games Officiated': list(referee_counts.values())
    })
    # Include all referees
    for ref in ref_available_df['ref_name']:
        if ref not in referee_counts:
            ref_counts_df = pd.concat([
                ref_counts_df,
                pd.DataFrame({'Referee': [ref], 'Games Officiated': [0]})
            ], ignore_index=True)
    
    # Save output
    output_file = f"schedules/schedule_case_{case_number}.xlsx"
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        schedule_df.to_excel(writer, sheet_name="Schedule", index=False)
        group_df.to_excel(writer, sheet_name="Groupings", index=False)
        ref_counts_df.to_excel(writer, sheet_name="Referee counts", index=False)
    
    return len(unscheduled)

# Process all cases
for i in range(1, 6):
    unscheduled_count = schedule_matches(i)
    print(f"Generated schedules/schedule_case_{i}.xlsx with {unscheduled_count} unscheduled matches.")

print("Scheduling complete. Check the 'schedules' folder for output files.")

Case 1: All matches scheduled successfully.
Case 1: Last match scheduled on day 11 (mon)
Case 1: Objective value = 11.1080 (makespan=11, penalty=0.0000, balance_penalty=10.8000)
Generated schedules/schedule_case_1.xlsx with 0 unscheduled matches.
Case 2: All matches scheduled successfully.
Case 2: Last match scheduled on day 15 (fri)
Case 2: Objective value = 15.5920 (makespan=15, penalty=0.4000, balance_penalty=19.2000)
Generated schedules/schedule_case_2.xlsx with 0 unscheduled matches.
Case 3: All matches scheduled successfully.
Case 3: Last match scheduled on day 12 (tue)
Case 3: Objective value = 12.4080 (makespan=12, penalty=0.2000, balance_penalty=20.8000)
Generated schedules/schedule_case_3.xlsx with 0 unscheduled matches.
Case 4: All matches scheduled successfully.
Case 4: Last match scheduled on day 15 (fri)
Case 4: Objective value = 15.5400 (makespan=15, penalty=0.4000, balance_penalty=14.0000)
Generated schedules/schedule_case_4.xlsx with 0 unscheduled matches.
Case 5: All 

### optimal solution (same logic as version_generate_groups)

In [ ]:
import pandas as pd
from gurobipy import Model, GRB, quicksum
from itertools import combinations
import os

# Create 'schedules' folder if it doesn't exist
if not os.path.exists("schedules"):
    os.makedirs("schedules")

# Weekday mapping (5-day cycle: Day 1 = Monday, Day 5 = Friday, Day 6 = Monday)
WEEKDAYS = {1: "mon", 2: "tue", 3: "wed", 4: "thur", 0: "fri"}

def get_weekday(day):
    """Map day number to weekday name."""
    return WEEKDAYS[int(day) % 5]

def get_week_number(day):
    """Determine the week number (1-based) for a given day."""
    return (int(day) - 1) // 7 + 1

def optimize_schedule(case_number):
    """Optimize match schedule for a given case using Gurobi."""
    # Load data from case_{i}.xlsx
    input_file = f"cases/case_{case_number}.xlsx"
    try:
        group_df = pd.read_excel(input_file, sheet_name="group")
        ref_available_df = pd.read_excel(input_file, sheet_name="ref_available")
        team_available_df = pd.read_excel(input_file, sheet_name="team_available")
        ref_conflicts_df = pd.read_excel(input_file, sheet_name="ref_conflicts")
    except Exception as e:
        print(f"Error loading case_{case_number}.xlsx: {e}")
        return

    # Process data
    # Referees and days (keep days as strings to match DataFrame columns)
    referees = ref_available_df['ref_name'].tolist()
    days = [col for col in ref_available_df.columns if col != 'ref_name']
    fields = list(range(4))

    # Referee availability (keep original values: 0, 0.5, 1)
    ref_available = ref_available_df.set_index('ref_name')[days].T

    # Referee conflicts
    ref_conflicts = ref_conflicts_df.set_index('ref_name').to_dict('index')
    for ref in ref_conflicts:
        ref_conflicts[ref] = {group: int(value) for group, value in ref_conflicts[ref].items()}

    # Team availability
    team_week_avail = team_available_df.set_index('team_name')[['mon', 'tue', 'wed', 'thur', 'fri']].to_dict('index')

    # Groupings
    grouped_teams = {}
    for group, g_df in group_df.groupby('group'):
        grouped_teams[group] = list(g_df['team_name'])
    
    # All teams
    teams = [team for group in grouped_teams.values() for team in group]

    # Valid days (exclude days 31-35 where all referees are unavailable)
    valid_days = [d for d in days if int(d) < 31 or int(d) > 35]
    day_to_weekday = {d: get_weekday(d) for d in valid_days}

    # Generate matches (single round-robin per group)
    matches = []
    for group, team_list in grouped_teams.items():
        if len(team_list) < 2:
            print(f"⚠️ Warning: Group {group} has only {len(team_list)} teams, no matches generated.")
            continue
        for t1, t2 in combinations(team_list, 2):
            matches.append((group, t1, t2))

    # Initialize Gurobi model
    m = Model(f"volleyball_schedule_case_{case_number}")
    m.setParam('OutputFlag', 0)

    # Decision variables: x[group, t1, t2, d, f, r] = 1 if match (t1 vs t2) is scheduled
    x = {}
    for group, t1, t2 in matches:
        for d in valid_days:
            wd = day_to_weekday[d]
            if team_week_avail.get(t1, {}).get(wd, 0) == 1 and team_week_avail.get(t2, {}).get(wd, 0) == 1:
                for f in fields:
                    for r in referees:
                        if ref_available.loc[d, r] >= 0.5 and ref_conflicts[r].get(group, 0) == 1:
                            x[(group, t1, t2, d, f, r)] = m.addVar(vtype=GRB.BINARY, name=f"x_{group}_{t1}_{t2}_{d}_{f}_{r}")

    # Makespan: last day used
    makespan = m.addVar(vtype=GRB.INTEGER, name="makespan")

    # Referee games and deviation
    referee_games = {r: m.addVar(vtype=GRB.INTEGER, name=f"ref_games_{r}") for r in referees}
    total_matches = len(matches)
    avg_games = total_matches / len(referees) if referees else 0
    deviation_plus = {r: m.addVar(vtype=GRB.CONTINUOUS, name=f"dev_plus_{r}") for r in referees}
    deviation_minus = {r: m.addVar(vtype=GRB.CONTINUOUS, name=f"dev_minus_{r}") for r in referees}

    # Constraint: Calculate referee games
    for r in referees:
        m.addConstr(
            referee_games[r] == quicksum(
                x.get((group, t1, t2, d, f, rr), 0)
                for (g, t1, t2, d, f, rr) in x if rr == r
            ),
            name=f"ref_games_{r}"
        )

    # Constraint: Deviation from average games
    for r in referees:
        m.addConstr(referee_games[r] == avg_games + deviation_plus[r] - deviation_minus[r])

    # Constraints
    # 1. Each match scheduled exactly once
    for group, t1, t2 in matches:
        m.addConstr(
            quicksum(x.get((group, t1, t2, d, f, r), 0)
                     for d in valid_days for f in fields for r in referees
                     if (group, t1, t2, d, f, r) in x)
            == 1,
            name=f"match_{group}_{t1}_{t2}"
        )

    # 2. At most one match per field per day
    for d in valid_days:
        for f in fields:
            m.addConstr(
                quicksum(x.get((group, t1, t2, dd, ff, r), 0)
                         for (group, t1, t2, dd, ff, r) in x if dd == d and ff == f)
                <= 1,
                name=f"field_{d}_{f}"
            )

    # 3. At most one match per referee per day
    for d in valid_days:
        for r in referees:
            m.addConstr(
                quicksum(x.get((group, t1, t2, dd, f, rr), 0)
                         for (group, t1, t2, dd, f, rr) in x
                         if dd == d and rr == r)
                <= 1,
                name=f"ref_{r}_day_{d}"
            )

    # 4. At most one match per team per day
    for t in teams:
        for d in valid_days:
            m.addConstr(
                quicksum(x.get((group, t1, t2, dd, f, r), 0)
                         for (group, t1, t2, dd, f, r) in x
                         if dd == d and (t1 == t or t2 == t))
                <= 1,
                name=f"team_{t}_day_{d}"
            )

    # 5. At most two matches per team per week
    weeks = sorted(set(get_week_number(d) for d in valid_days))
    for t in teams:
        for w in weeks:
            week_days = [d for d in valid_days if get_week_number(d) == w]
            m.addConstr(
                quicksum(x.get((group, t1, t2, dd, f, r), 0)
                         for (group, t1, t2, dd, f, r) in x
                         if dd in week_days and (t1 == t or t2 == t))
                <= 2,
                name=f"team_{t}_week_{w}"
            )

    # 6. Makespan constraint
    for key in x:
        group, t1, t2, d, f, r = key
        m.addConstr(x[key] * int(d) <= makespan, name=f"makespan_{group}_{t1}_{t2}_{d}")

    # Objective: Minimize makespan + penalties
    penalty = quicksum(0.1 * x[key] for key in x if ref_available.loc[key[3], key[5]] == 0.5)
    balance_penalty = quicksum(deviation_plus[r] + deviation_minus[r] for r in referees)
    m.setObjective(makespan + penalty + 0.01 * balance_penalty, GRB.MINIMIZE)

    # Optimize
    m.optimize()

    # Output results
    if m.status == GRB.OPTIMAL:
        result = [(group, t1, t2, int(d), f, r)
                  for (group, t1, t2, d, f, r), var in x.items() if var.X > 0.5]
        df = pd.DataFrame(result, columns=["Group", "Team1", "Team2", "Day", "Field", "Referee"])
        df["Match"] = df["Team1"] + " vs " + df["Team2"]
        schedule_df = df[["Day", "Group", "Field", "Match", "Referee"]].sort_values(by=["Day", "Field"])

        # Referee counts
        referee_counts = {r: 0 for r in referees}
        half_availability_count = 0
        for _, row in df.iterrows():
            referee_counts[row["Referee"]] += 1
            if ref_available.loc[str(row["Day"]), row["Referee"]] == 0.5:
                half_availability_count += 1

        # Calculate objective components
        makespan_value = int(makespan.X)
        penalty_value = 0.1 * half_availability_count
        balance_penalty_value = sum(
            abs(referee_counts[r] - avg_games) for r in referees
        )
        objective_value = makespan_value + penalty_value + 0.01 * balance_penalty_value

        referee_counts_df = pd.DataFrame.from_dict(
            referee_counts, orient='index', columns=["Games Officiated"]
        )
        referee_counts_df.index.name = "Referee"
        referee_counts_df = referee_counts_df.reset_index()

        # Save to Excel
        output_file = f"schedules/schedule_case_{case_number}.xlsx"
        with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
            schedule_df.to_excel(writer, sheet_name="Schedule", index=False)
            group_df.to_excel(writer, sheet_name="Groupings", index=False)
            referee_counts_df.to_excel(writer, sheet_name="Referee Counts", index=False)

        print("✅ 排程成功！最晚比賽日：", int(makespan.X))

        print(f"Case {case_number}: Objective value = {objective_value:.4f} "
              f"(makespan={makespan_value}, penalty={penalty_value:.4f}, balance={balance_penalty_value:.4f})")
        print(f"📤 已输出：{output_file}")
    else:
        print(f"❌ 未能找到可行程方案 for case_{case_number}")

# Process all cases
for i in range(1, 6):
    print(f"\nProcessing case_{i}.xlsx")
    optimize_schedule(i)


Processing case_1.xlsx
✅ 排程成功！最晚比賽日： 10
Case 1: Objective value = 10.1840 (makespan=10, penalty=0.0000, balance=18.4000)
📤 已输出：schedules/schedule_case_1.xlsx

Processing case_2.xlsx
✅ 排程成功！最晚比賽日： 14
Case 2: Objective value = 14.6480 (makespan=14, penalty=0.4000, balance=24.8000)
📤 已输出：schedules/schedule_case_2.xlsx

Processing case_3.xlsx
✅ 排程成功！最晚比賽日： 11
Case 3: Objective value = 11.2760 (makespan=11, penalty=0.0000, balance=27.6000)
📤 已输出：schedules/schedule_case_3.xlsx

Processing case_4.xlsx
✅ 排程成功！最晚比賽日： 12
Case 4: Objective value = 12.4800 (makespan=12, penalty=0.2000, balance=28.0000)
📤 已输出：schedules/schedule_case_4.xlsx

Processing case_5.xlsx
✅ 排程成功！最晚比賽日： 15
Case 5: Objective value = 15.3200 (makespan=15, penalty=0.0000, balance=32.0000)
📤 已输出：schedules/schedule_case_5.xlsx
